# Dataset Merge IDEA

- history data 기준 watch merge 작업
- watch 만 있는 아이템들에 대한 row를 history data 붙임
- 이후 meta,meta_plus,profile data merge 작업

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
history = pd.read_csv('../../data/history_data.csv')
history.drop_duplicates(subset=['profile_id','log_time','album_id'],inplace=True) 
# row = 1005651 rows × 8 columns
# after drop duplicates 899021 rows × 8 columns
# 106630 개 row 중복

### Watch에 대해서도 동일한 작업 수행 (duplicate drop, feature selection)

In [3]:
watch = pd.read_csv('../../data/watch_e_data.csv')
watch.drop_duplicates(subset=['profile_id','log_time','album_id'],inplace=True) 
# watch.drop(labels=['log_time','act_target_dtl'],axis=1,inplace=True)  # column 드랍. 따로 처리할 결측치 없음

In [7]:
print('각 데이터 수:',len(history), len(watch))
print('history 만 있는 item 수:',len(set(history.album_id)-set(watch.album_id)), 'watch 만 있는 item 수:',len(set(watch.album_id)-set(history.album_id)))
print('history 만 있는 user 수:',len(set(history.profile_id)-set(watch.profile_id)), 'watch 만 있는 item 수:',len(set(watch.profile_id)-set(history.profile_id)))

각 데이터 수: 899252 800632
history 만 있는 item 수: 365 watch 만 있는 item 수: 31
history 만 있는 user 수: 653 watch 만 있는 item 수: 0


- watch 데이터에만 있는 item row들을 history 추가

In [19]:
print('history 추가될 row 수:', len(watch[watch['album_id'].isin(set(watch.album_id)-set(history.album_id))]))

history 추가될 row 수: 47


In [20]:
unique_item_watch_df = watch[watch['album_id'].isin(set(watch.album_id)-set(history.album_id))][['profile_id','ss_id','log_time','act_target_dtl','album_id']]
dataset = pd.concat([history,unique_item_watch_df], axis=0)

In [21]:
# 재확인
print('각 데이터 수:',len(dataset), len(watch))
print('history 만 있는 item 수:',len(set(dataset.album_id)-set(watch.album_id)), 'watch 만 있는 item 수:',len(set(watch.album_id)-set(dataset.album_id)))
print('history 만 있는 user 수:',len(set(dataset.profile_id)-set(watch.profile_id)), 'watch 만 있는 item 수:',len(set(watch.profile_id)-set(dataset.profile_id)))

각 데이터 수: 899299 800632
history 만 있는 item 수: 365 watch 만 있는 item 수: 0
history 만 있는 user 수: 653 watch 만 있는 item 수: 0


# 1. watch_e 가 history merge

In [114]:
dataset.to_csv('./history_dataset.csv') 
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 815873 entries, 0 to 815872
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   profile_id       815873 non-null  int64  
 1   ss_id            815873 non-null  int64  
 2   log_time         815873 non-null  int64  
 3   act_target_dtl   815873 non-null  object 
 4   album_id         815873 non-null  int64  
 5   watch_time       800632 non-null  float64
 6   total_time       800632 non-null  float64
 7   continuous_play  815873 non-null  object 
 8   payment          564 non-null     float64
 9   short_trailer    15241 non-null   object 
dtypes: float64(3), int64(4), object(3)
memory usage: 62.2+ MB


# 2. Meta Data merge 

In [22]:
# # 메타데이터 병합 
meta = pd.read_csv('../../data/meta_data.csv') # 42602 rows
meta_p = pd.read_csv('../../data/meta_data_plus.csv') # 767948 rows
# meta_merge = pd.merge(meta,meta_p,how='inner',on='album_id') #### meta 정보 최종 집합체 : 832356 rows
# data_w_meta = pd.merge(dataset,meta,how='inner',on='album_id')

/var/folders/tb/j9qmrzv54k73r5l100jxqll80000gn/T/ipykernel_54770/1435218993.py:2: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  meta = pd.read_csv('../../data/meta_data.csv') # 42602 rows


## Meta_plus 데이터 proprocess 

In [23]:
meta_p['keyword_type'].nunique()

1101

In [24]:
pd.set_option('display.max_rows',500)
# meta_p['keyword_name'].value_counts() # 1개 키워드 개수는 30개 가량뿐이다. 일단 keyword를 신뢰할 만하다고 판단.

# 한편, 아주 비슷한 keyword끼리는 encoding 전에 같은 태그로 통일할 필요가 있음. (ex. 치아 구강 ↔ 치아,  용감함 ↔ 용감함4)
# 이걸 확인하기 위해서, sort로 1062개를 정렬해본다. (휴먼 러닝을 피할 수가 없음)

key_count = pd.DataFrame(meta_p['keyword_name'].value_counts().reset_index(drop=False).sort_values(by='index'))
# key_count

### 확인해 봐야 하는 (합쳐야 하는 단어 목록)
5세(05010403) > 5세 추천(050b0102)    
6세 > 6세 추천(050b0103)    
7세 > 7세 추천    
비교 < 비교하기    
안데르센 > 안데르센상    
야외놀이 > 야외놀이안전 (확인필요)    
언어1, 언어2 (확인필요)     
언어지능1, 언어지능2    
용감함 > 용감함 4    
의사소통 < 의사소통(Communication)    
자신감 < 자신감(Confidence)    
장선혜 > 장선혜(작화)    
최숙희 > 최숙희(작화)    
캐리 < 캐리TV
> 숫자가 더 적은 쪽을 변경하기로 한다. 

In [25]:
# 태그 코드는 떨어트릴 거지만 그래도 같이 변경해주자.

meta_p.replace({'keyword_name' : '안데르센상'}, '안데르센',inplace=True)
meta_p.replace({'keyword_name' : '용감함 4'}, '용감함',inplace=True)
meta_p.replace({'keyword_name' : '의사소통'}, '의사소통(Communication)',inplace=True)
meta_p.replace({'keyword_name' : '자신감'}, '자신감(Confidence)',inplace=True)
meta_p.replace({'keyword_name' : '장선혜'}, '장선혜(작화)',inplace=True)
meta_p.replace({'keyword_name' : '최숙희'}, '최숙희(작화)',inplace=True)
meta_p.replace({'keyword_name' : '캐리'}, '캐리TV',inplace=True)

meta_p.replace({'keyword_type' : '05010a01'}, '0501090d',inplace=True)
meta_p.replace({'keyword_type' : '0509071c'}, '05090708',inplace=True)
meta_p.replace({'keyword_type' : '05030908'}, '050c1002',inplace=True)
meta_p.replace({'keyword_type' : '05070607'}, '050c1006',inplace=True)
meta_p.replace({'keyword_type' : '05010907'}, '05010914',inplace=True)
meta_p.replace({'keyword_type' : '05010909'}, '05010915',inplace=True)
meta_p.replace({'keyword_type' : '05080409'}, '0501080a',inplace=True)

In [26]:
meta_p['keyword_type'].nunique()
#제대로 정리됨

1094

# 3. (meta+plus)data Merge 작업 

In [27]:
# sub_title 리스트 담기
meta_df_sub_title_list_dict= {}

for idx, row in meta.iterrows():
    if meta_df_sub_title_list_dict.get(row['album_id'], None) == None:
        meta_df_sub_title_list_dict[row['album_id']] = [row['sub_title']]
    elif row['sub_title'] not in meta_df_sub_title_list_dict[row['album_id']]:
        meta_df_sub_title_list_dict[row['album_id']] = meta_df_sub_title_list_dict[row['album_id']]+[row['sub_title']]
        
print('전체 컨텐츠 개수:',len(set(meta.album_id)),'딕셔너리 담긴 아이템 개수:',len(meta_df_sub_title_list_dict))        

meta_df_prepro_sub_title = meta.copy()
meta_df_prepro_sub_title['sub_title'] = meta_df_prepro_sub_title['album_id'].apply(lambda x: meta_df_sub_title_list_dict.get(x, None))

전체 컨텐츠 개수: 39875 딕셔너리 담긴 아이템 개수: 39875


In [28]:
# 전체 중복 삭제하기
meta_df_prepro_sub_title = meta_df_prepro_sub_title.drop_duplicates(subset=['album_id'], ignore_index=True)

print('전체 데이터수와 album_id 개수 동일 확인:', len(meta_df_prepro_sub_title), len(meta_df_prepro_sub_title.album_id.unique()))

전체 데이터수와 album_id 개수 동일 확인: 39875 39875


In [29]:
# 메타 플러스 데이터 keyword(keyword_name, type)리스트 담기
meta_plus_type_name = meta_p.copy()

In [30]:
type_to_name = {}

for idx, row in meta_p.iterrows():
    type_to_name[row['keyword_type']] = row['keyword_name']
    
meta_plus_type_name['keyword_name'] = meta_plus_type_name['keyword_type'].apply(lambda x: type_to_name[x])    

name_to_type = {}
for idx, row in meta_p.iterrows():
    name_to_type[row['keyword_name']] = row['keyword_type']
    
meta_plus_type_name['keyword_type'] = meta_plus_type_name['keyword_name'].apply(lambda x: name_to_type[x])
print('keyword_name, keyword_type 개수 확인:',len(set(meta_plus_type_name.keyword_name)), len(set(meta_plus_type_name.keyword_type)))
print('앨범 당 키워드 중복 여부 확인:',len(meta_p),len(meta_p.drop_duplicates(subset=['album_id','keyword_type'])))

keyword_name, keyword_type 개수 확인: 1055 1055
앨범 당 키워드 중복 여부 확인: 767948 766943


In [31]:
meta_plus_df_prepro_keyword = meta_plus_type_name.copy()

In [32]:
# meta plus data type name 동일하게 만들기

meta_df_keyword_name_list_dict= {}
meta_df_keyword_type_list_dict= {}

for idx, row in meta_plus_df_prepro_keyword.iterrows():
    if meta_df_keyword_name_list_dict.get(row['album_id'], None) == None:
        meta_df_keyword_name_list_dict[row['album_id']] = [row['keyword_name']]
    elif row['keyword_name'] not in meta_df_keyword_name_list_dict[row['album_id']]:
        meta_df_keyword_name_list_dict[row['album_id']] = meta_df_keyword_name_list_dict[row['album_id']]+[row['keyword_name']]

for idx, row in meta_plus_df_prepro_keyword.iterrows():
    if meta_df_keyword_type_list_dict.get(row['album_id'], None) == None:
        meta_df_keyword_type_list_dict[row['album_id']] = [row['keyword_type']]
    elif row['keyword_type'] not in meta_df_keyword_type_list_dict[row['album_id']]:
        meta_df_keyword_type_list_dict[row['album_id']] = meta_df_keyword_type_list_dict[row['album_id']]+[row['keyword_type']]        

meta_plus_df_prepro_keyword['keyword_name'] = meta_plus_df_prepro_keyword['album_id'].apply(lambda x: meta_df_keyword_name_list_dict.get(x, None))
meta_plus_df_prepro_keyword['keyword_type'] = meta_plus_df_prepro_keyword['album_id'].apply(lambda x: meta_df_keyword_type_list_dict.get(x, None))

meta_plus_df_keyword = meta_plus_df_prepro_keyword.drop_duplicates(subset=['album_id'],ignore_index=True).drop(columns=['keyword_value'])

In [33]:
# meta data merge
all_meta_df = pd.merge(meta_df_prepro_sub_title, meta_plus_df_keyword, how='left',on='album_id')
all_meta_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39875 entries, 0 to 39874
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   album_id      39875 non-null  int64  
 1   title         39875 non-null  object 
 2   sub_title     39875 non-null  object 
 3   genre_large   39875 non-null  object 
 4   genre_mid     39875 non-null  object 
 5   genre_small   12279 non-null  object 
 6   country       31773 non-null  object 
 7   run_time      39875 non-null  int64  
 8   onair_date    5208 non-null   float64
 9   cast_1        26309 non-null  object 
 10  cast_2        21217 non-null  object 
 11  cast_3        15885 non-null  object 
 12  cast_4        11980 non-null  object 
 13  cast_5        6146 non-null   object 
 14  cast_6        2540 non-null   object 
 15  cast_7        743 non-null    object 
 16  keyword_type  39875 non-null  object 
 17  keyword_name  39875 non-null  object 
dtypes: float64(1), int64(2), o

In [34]:
# all_meta_df.to_csv('./all_meta_df.csv')

In [35]:
data1 = pd.merge(dataset, all_meta_df, how='left',on='album_id')
# 아래처럼 병합하면 data2 의 column명에 _x, _y같은 값이 붙어있는 문제 발생 
# data2 = pd.merge(data1,history, how='left', on=['ss_id','album_id'])


# payment 정보 더하기  
'''
history에서 [payment/log_time/album_id]만 가진 dataframe 만들기
data1에 left join 하는데, 기준을 album_id, log_time 
== 같은 시점에 유료였던 것은 유료로 값이 들어가겠죠? (payment는 시간을 고려할수밖에 없습니다)
ㄴ (보류) 방금 진수님이 이야기한 걸 바탕으로는 : 유저 개개인도 payment를 따지는 데 중요한 고려 요소인 것 같아요? <추후 확인>
   유저가 한번 결제한 영상 > 
'''

'\nhistory에서 [payment/log_time/album_id]만 가진 dataframe 만들기\ndata1에 left join 하는데, 기준을 album_id, log_time \n== 같은 시점에 유료였던 것은 유료로 값이 들어가겠죠? (payment는 시간을 고려할수밖에 없습니다)\nㄴ (보류) 방금 진수님이 이야기한 걸 바탕으로는 : 유저 개개인도 payment를 따지는 데 중요한 고려 요소인 것 같아요? <추후 확인>\n   유저가 한번 결제한 영상 > \n'

## payment one-hot-encoding

In [38]:
# 딕셔너리 형태로 payment 있는 앨범당 값 부여
payment_dict = data1[data1['payment']>0][['album_id','payment']].set_index('album_id').to_dict()
data1['payment'] = data1['album_id'].apply(lambda x: 1 if x in list(payment_dict['payment']) else 0)

### payment 기준 mp100 구하기

In [39]:
# payment MP@100 목록
# 주의 사항: 유료무료유료뮤료 였던 아이템에 대한 파악 불가, 기간제 아이템 확인 불가

payment_mp100 = data1[data1['payment']==1].groupby('album_id').count()['payment'].reset_index().rename(columns={'payment':'cnt'}).sort_values(by='cnt',ascending=False, ignore_index=True).head(100)
payment_mp100

,album_id,cnt
0,264,1618
1,178,1614
2,185,1595
3,2196,1496
4,175,1386
5,33,1062
6,3954,985
7,309,973
8,181,969
9,381,940


# 4. profile 파일 merge

In [40]:
# 파일 로드
profile_df = pd.read_csv('../../data/profile_data.csv')

# keyword 한글 인덱싱
pr_interest_keyword_dict = {'P01':'과학기술',
                           'P02':'정서/사회성',
                           'P03':'자연탐구',
                           'P04':'바른생활/안전',
                           'P05':'활동/운동',
                           'P06':'음악예술',
                           'P07':'언어논리',
                           'P08':'수리논리'}
ch_interest_keyword_dict = {'K01':'노래/율동',
                           'K02':'동물/식물',
                           'K03':'동화',
                           'K04':'만들기',
                           'K05':'숫자/계산',
                           'K06':'외국어',
                           'K07':'친구/사람',
                           'K08':'탈것/기계',
                           'K09':'활동/운동'}

pr_list = ['pr_interest_keyword_cd_1','pr_interest_keyword_cd_2','pr_interest_keyword_cd_3']
ch_list = ['ch_interest_keyword_cd_1','ch_interest_keyword_cd_2','ch_interest_keyword_cd_3']
for pr in pr_list:
    profile_df[pr] = profile_df[pr].apply(lambda x: pr_interest_keyword_dict.get(x, np.NaN))
for ch in ch_list:
    profile_df[ch] = profile_df[ch].apply(lambda x: ch_interest_keyword_dict.get(x, np.NaN))

In [41]:
print('merge 가능 확인:', set(data1.profile_id) - set(profile_df.profile_id), set(profile_df.profile_id) - set(data1.profile_id))

merge 가능 확인: set() set()


# 5. 최종 merge (watch_e, history, meta, meta_plus, profile)

In [43]:
# 최종 데이터셋
data = pd.merge(data1,profile_df, how='left', on='profile_id')
data.to_csv('hitory_data.csv')

## 잊지 말아야 할 것
> history의 nunique한 아이템 수 : 대략 2만건   
mata의 nunique한 아이템 수 : 대략 4만건   
== 2달간의 소비데이터에 등장조차 하지 않은 아이템이 2만개나 있다

> **todo** 한 달씩 분할해서 / 첫 달에 등장한 아이템이 두번째 달에 많이 등장하는지 확인해보면 될 것 

# feature encoding


. . . loading

# Neg Samplig

시청하지 않은 것에 대해서 [시청 시간 feature]를 넣어줘야 함     
ㄴ 시청하지 않은 영상이 너무 많아서 네거티브 샘플링을 하면 편중/과적합이 된다     
ㄴ 관련 논문이나 연구가 있을 법 한데 (찾아보기)


진수's idea
어차피 랜덤하게 샘플링하니까 시청 시간도 랜덤하게 값을 넣어보자 (ex. 해당 아이템의 평균 시청시간)
*세션이 해당 유저가 들어온 시간이라면, 그 시간에 대해서는 로그*
유저가 연속해서 재생한 것과, 종료한 다음 재생한 것에 대해 기록에 차이를 줘야 하지 않나?